<a href="https://colab.research.google.com/github/agasparsovero-droid/modelo-estudiantes-web/blob/main/notas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)



Mounted at /content/drive


In [ ]:
# Instala dependencias si hiciera falta (descomenta si lo necesitas)
# !pip install tensorflow==2.12.0 scikit-learn pandas matplotlib joblib --quiet

import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, mean_squared_error
import joblib
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
# 2) Cargar el Excel (ajusta la ruta a tu archivo en Drive)
path = "/content/drive/MyDrive/datos2.xlsx"   # <-- ajusta a donde esté tu archivo
assert os.path.exists(path), f"No encontré {path} — revisa la ruta en Drive."

df = pd.read_excel(path)
# Normalizar nombres de columnas (sin espacios ni símbolos)
df.columns = df.columns.str.strip().str.replace(r'[^0-9a-zA-Z_]+', '_', regex=True)
print("Columnas:", df.columns.tolist())
df.head()

AssertionError: No encontré /content/drive/MyDrive/datos2.xlsx — revisa la ruta en Drive.

In [ ]:
# 3) Inspección rápida y limpieza
# Asegúrate que tu columna objetivo se llama exactamente Promedio_Notas (si no, ajusta)
assert "Promedio_Notas" in df.columns, "Asegúrate que el Excel tiene la columna 'Promedio_Notas'"

# Guardar IDs si las necesitas luego
if "ID_Estudiante" in df.columns:
    ids = df["ID_Estudiante"].copy()

# Ver tipos y valores únicos de las columnas categóricas
for col in ["Nivel", "Situacion_Socioeconomica", "Motivacion", "Responsabilidad"]:
    if col in df.columns:
        print(col, "->", df[col].dropna().unique()[:20])


Nivel -> ['Secundaria' 'Primaria']
Situacion_Socioeconomica -> ['Media' 'Media-Baja' 'Media-Alta' 'Baja' 'Alta']
Motivacion -> ['Alta' 'Muy Alta' 'Baja' 'Media' 'Muy Baja']
Responsabilidad -> ['Alta' 'Muy Alta' 'Baja' 'Media']


In [ ]:
# 4) Mapear variables categóricas (ordinales). Ajusta los mapeos si tus etiquetas son distintas.
# Mapeos (usar keys en minúscula)
nivel_map = {"primaria": 0, "secundaria": 1}

situ_map = {
    "baja": 1,
    "media baja": 2,
    "media": 3,
    "media alta": 4,
    "alta": 5
}

motiv_map = {
    "muy baja": 1,
    "baja": 2,
    "media": 3,
    "alta": 4,
    "muy alta": 5
}

resp_map = {
    "baja": 1,
    "media": 2,
    "alta": 3,
    "muy alta": 4
}

# Función segura de mapeo
def map_lower(series, mapping, fill_with=np.nan):
    s = series.fillna("").astype(str).str.lower().str.strip()
    mapped = s.map(mapping)
    if pd.isna(fill_with):
        return mapped
    # Calculate mode only if needed for fillna
    if fill_with == 'mode':
        mode_val = mapped.mode()
        if not mode_val.empty:
            return mapped.fillna(mode_val[0])
        else:
            # If mode is empty (all NaNs or no valid values after mapping), fill with a default
            return mapped.fillna(np.nan) # Or a specific default number like 0 or median
    return mapped.fillna(fill_with)

# Aplicar mapeos (si la columna no existe, la ignoramos)
if "Nivel" in df.columns:
    df["Nivel_num"] = map_lower(df["Nivel"], nivel_map, fill_with=0)

if "Situacion_Socioeconomica" in df.columns:
    # Use 'mode' as a strategy to fill with the calculated mode
    df["Situacion_num"] = map_lower(df["Situacion_Socioeconomica"], situ_map, fill_with='mode')

if "Motivacion" in df.columns:
    # Use 'mode' as a strategy to fill with the calculated mode
    df["Motivacion_num"] = map_lower(df["Motivacion"], motiv_map, fill_with='mode')


if "Responsabilidad" in df.columns:
    # Use 'mode' as a strategy to fill with the calculated mode
    df["Responsabilidad_num"] = map_lower(df["Responsabilidad"], resp_map, fill_with='mode')

In [ ]:
# 5) Seleccionar variables predictoras y objetivo
# Ajusta la lista de features si tus columnas tienen otros nombres
features = []
for c in ["Nivel_num", "Horas_Estudio_Semanal", "Asistencia_pct", "Asistencia", "Situacion_num", "Motivacion_num", "Responsabilidad_num"]:
    # permitimos tanto "Asistencia (%)" como "Asistencia" o "Asistencia_pct" según cómo quedó tras normalizar nombres
    if c in df.columns and c not in features:
        features.append(c)

# Si 'Asistencia' vino con "%" y fue normalizada a "Asistencia_pct" o similar, revisa la lista estampada arriba.
print("Features usados:", features)

# Si no detectamos Horas_Estudio_Semanal porque el nombre es distinto, intenta un fallback:
if "Horas_Estudio_Semanal" not in features:
    for col in df.columns:
        if "Horas" in col and "Estudio" in col:
            features.append(col)
            print("Usando columna encontrada ->", col)
            break

X = df[features].copy()
y = df["Promedio_Notas"].astype(float).copy()

# Imputación simple para NaNs
num_imputer = SimpleImputer(strategy="median")
X_numeric_cols = X.select_dtypes(include=[np.number]).columns.tolist()
X[X_numeric_cols] = num_imputer.fit_transform(X[X_numeric_cols])

# If there are still non-numeric columns after mapping, identify them
non_numeric_cols = X.select_dtypes(exclude=[np.number]).columns.tolist()
if non_numeric_cols:
    print(f"Warning: The following columns are still non-numeric: {non_numeric_cols}")
    # You might want to inspect these columns to understand why they weren't mapped

print("Tamaño final X, y:", X.shape, y.shape)

Features usados: ['Nivel_num', 'Horas_Estudio_Semanal', 'Situacion_num', 'Motivacion_num', 'Responsabilidad_num']
Tamaño final X, y: (200, 5) (200,)


Now that the data is processed, let's split it into training and testing sets and scale the features.

In [ ]:
# 6) Dividir los datos (Train/Test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Tamaño X_train, X_test:", X_train.shape, X_test.shape)
print("Tamaño y_train, y_test:", y_train.shape, y_test.shape)

Tamaño X_train, X_test: (160, 5) (40, 5)
Tamaño y_train, y_test: (160,) (40,)


Next, we will scale the features using `StandardScaler`. This is a common practice for many machine learning models.

In [ ]:
# 7) Escalar las features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Guardar el scaler para usarlo en predicciones futuras
joblib.dump(scaler, "scaler.pkl")

print("Features escalados correctamente.")

Features escalados correctamente.


Now that the data is scaled, let's define the neural network model.

Let's train the model using the scaled training data. We will use Early Stopping to prevent overfitting.

Let's evaluate the model on the test set to see how well it generalizes to unseen data.

Now we can save the trained model and the scaler.

Now that the data is scaled, let's define the neural network model.

In [ ]:
# 8) Definir el modelo (ajusta según tus necesidades)
# Puedes experimentar con diferentes arquitecturas, capas, y unidades
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.2), # Regularización para evitar overfitting
    Dense(32, activation='relu'),
    Dense(1) # Capa de salida con 1 unidad para regresión (predicción de un valor continuo)
])

# Compilar el modelo
# Usamos 'mse' (Mean Squared Error) como función de pérdida y 'mae' (Mean Absolute Error) como métrica
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,497 (9.75 KB)

 Trainable params: 2,497 (9.75 KB)

 Non-trainable params: 0 (0.00 B)

Let's train the model using the scaled training data. We will use Early Stopping to prevent overfitting.

In [ ]:
# 9) Entrenar el modelo
# Define Callbacks
# EarlyStopping: para detener el entrenamiento si la métrica de validación no mejora
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# ModelCheckpoint: para guardar el mejor modelo basado en la métrica de validación
model_checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_loss')


history = model.fit(
    X_train_scaled, y_train,
    epochs=100, # Número máximo de épocas (EarlyStopping lo detendrá antes si es necesario)
    batch_size=32, # Tamaño del batch
    validation_split=0.2, # Usar 20% de los datos de entrenamiento para validación
    callbacks=[early_stopping, model_checkpoint], # Aplicar los callbacks
    verbose=1 # Mostrar progreso
)

print("\nEntrenamiento finalizado.")

Epoch 1/100
1/4 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - loss: 243.1201 - mae: 15.3488

4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 76ms/step - loss: 238.8430 - mae: 15.2136 - val_loss: 267.2242 - val_mae: 16.1630
Epoch 2/100
1/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 209.8541 - mae: 14.2828

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 222.4565 - mae: 14.7019 - val_loss: 260.1428 - val_mae: 15.9461
Epoch 3/100
1/4 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 220.5677 - mae: 14.5775

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 220.5860 - mae: 14.6062 - val_loss: 253.1740 - val_mae: 15.7299
Epoch 4/100
1/4 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 217.6891 - mae: 14.5199

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 215.9605 - mae: 14.4656 - val_loss: 246.0451 - val_mae: 15.5054
Epoch 5/100
1/4 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 208.2210 - mae: 14.1927

...

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 3.5369 - mae: 1.4907 - val_loss: 3.3256 - val_mae: 1.5151
Epoch 96/100
1/4 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 5.0041 - mae: 1.8754

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 3.7465 - mae: 1.5600 - val_loss: 3.2963 - val_mae: 1.5080
Epoch 97/100
1/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 3.9545 - mae: 1.6511

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 3.6642 - mae: 1.5399 - val_loss: 3.2712 - val_mae: 1.4959
Epoch 98/100
1/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 2.6773 - mae: 1.3189

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 3.2040 - mae: 1.4110 - val_loss: 3.2514 - val_mae: 1.4802
Epoch 99/100
1/4 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 3.2773 - mae: 1.4654

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 3.2460 - mae: 1.4163 - val_loss: 3.2511 - val_mae: 1.4629
Epoch 100/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 2.9484 - mae: 1.3921 - val_loss: 3.2651 - val_mae: 1.4513

Entrenamiento finalizado.


Let's evaluate the model on the test set to see how well it generalizes to unseen data.

In [ ]:
# 10) Evaluar el modelo
loss, mae = model.evaluate(X_test_scaled, y_test, verbose=0)
print(f"Pérdida (MSE) en el conjunto de prueba: {loss:.2f}")
print(f"Error Absoluto Medio (MAE) en el conjunto de prueba: {mae:.2f}")

Pérdida (MSE) en el conjunto de prueba: 2.21
Error Absoluto Medio (MAE) en el conjunto de prueba: 1.24


Now we can save the trained model and the scaler.

In [ ]:
# 11) Guardar modelo y scaler
# Guardar modelo
model.save("/content/drive/MyDrive/modelo_estudiantes.h5")
print("Modelo guardado en formato H5")

# Guardar scaler en tu Drive
joblib.dump(scaler, "/content/drive/MyDrive/scaler_estudiantes.pkl")
print("Scaler guardado en Drive")

Modelo guardado en formato H5
Scaler guardado en Drive


In [ ]:
import joblib

# Guardar scaler en tu Drive
joblib.dump(scaler, "/content/drive/MyDrive/scaler_estudiantes.pkl")
print("Scaler guardado en Drive")

Scaler guardado en Drive


In [ ]:
# Conectar Colab con Drive
from google.colab import drive
drive.mount('/content/drive')

# Librerías necesarias
import numpy as np
import joblib
from tensorflow.keras.models import load_model
import os
import tensorflow as tf # Import tensorflow to access tf.keras.losses

# ---------------------------
# 1) Rutas de tus archivos
# ---------------------------
MODEL_PATH_KERAS = "/content/drive/MyDrive/modelo_estudiantes.keras" # Recommended Keras format
MODEL_PATH_H5 = "/content/drive/MyDrive/modelo_estudiantes.h5" # Old HDF5 format
SCALER_PATH = "/content/drive/MyDrive/scaler_estudiantes.pkl"

# ---------------------------
# 2) Cargar modelo y scaler
# ---------------------------
model = None
if os.path.exists(MODEL_PATH_KERAS):
    try:
        model = load_model(MODEL_PATH_KERAS)
        print(f"✅ Modelo cargado correctamente desde {MODEL_PATH_KERAS}")
    except Exception as e:
        print(f"Error loading .keras model: {e}")
        print("Attempting to load .h5 model instead.")

if model is None and os.path.exists(MODEL_PATH_H5):
    try:
        # Attempt to load .h5 model with custom_objects
        model = load_model(MODEL_PATH_H5, custom_objects={'mse': tf.keras.losses.MeanSquaredError, 'mae': tf.keras.metrics.MeanAbsoluteError})
        print(f"✅ Modelo cargado correctamente desde {MODEL_PATH_H5} with custom objects")
    except Exception as e:
        print(f"Error loading .h5 model: {e}")
        raise FileNotFoundError("Neither .keras nor .h5 model file could be loaded.")
elif model is None:
     raise FileNotFoundError("Neither .keras nor .h5 model file found in Google Drive.")


scaler = joblib.load(SCALER_PATH)

print("✅ Scaler cargado correctamente")

# ---------------------------
# 3) Simular un nuevo estudiante
# ---------------------------
# Orden de features used in training: ['Nivel_num', 'Horas_Estudio_Semanal', 'Situacion_num', 'Motivacion_num', 'Responsabilidad_num']
# Example: estudiante de secundaria (1), 8 horas de estudio,
# situación socioeconómica = media alta (4), motivación = alta (4), responsabilidad = muy alta (4)
# Make sure the number of features matches the model's input shape (5 features in this case based on X.shape)

nuevo_estudiante = np.array([[1, 8, 4, 4, 4]])

print("Datos de entrada (shape):", nuevo_estudiante.shape)
print("Datos de entrada:", nuevo_estudiante)


# ---------------------------
# 4) Escalar datos y predecir
# ---------------------------
nuevo_estudiante_scaled = scaler.transform(nuevo_estudiante)
prediccion = model.predict(nuevo_estudiante_scaled)

print("📘 Predicción del promedio estimado:", prediccion[0][0])

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Modelo cargado correctamente desde /content/drive/MyDrive/modelo_estudiantes.h5 with custom objects
✅ Scaler cargado correctamente
Datos de entrada (shape): (1, 5)
Datos de entrada: [[1 8 4 4 4]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
📘 Predicción del promedio estimado: 14.118146


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [ ]:
#DATO IMPORTANTE PARA PREDICCION MODELO YA ENTRENADO
nuevo_estudiante = np.array([[1, 13, 2, 2, 3]])

print("Datos de entrada (shape):", nuevo_estudiante.shape)
print("Datos de entrada:", nuevo_estudiante)


# ---------------------------
# 4) Escalar datos y predecir
# ---------------------------
nuevo_estudiante_scaled = scaler.transform(nuevo_estudiante)
prediccion = model.predict(nuevo_estudiante_scaled)

print("📘 Predicción del promedio estimado:", prediccion[0][0])

Datos de entrada (shape): (1, 5)
Datos de entrada: [[ 1 13  2  2  3]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
📘 Predicción del promedio estimado: 12.672867
